## Code Sources

- https://towardsdatascience.com/image-feature-extraction-using-pytorch-e3b327c3607a

- https://github.com/MnCSSJ4x/VR-MiniProject/blob/main/VR3b.ipynb

- https://colab.research.google.com/github/ashishpatel26/Awesome-Pytorch-Tutorials/blob/main/17.Pytorch%20Transfer%20learning%20with%20Caltech101.ipynb

## CUDA

In [2]:
from torch import cuda
from torch import device

In [3]:
'cuda:0' if cuda.is_available() else 'cpu'

'cuda:0'

In [4]:
device = device('cuda:0' if cuda.is_available() else 'cpu')

## AlexNet

In [5]:
from torchvision.models import alexnet

In [6]:
alexnet = alexnet(weights='AlexNet_Weights.DEFAULT')
print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Feature Extractor

In [7]:
!pip install torchsummary

In [8]:
from torch import nn
from torchsummary import summary

In [9]:
class FeatureExtractor(nn.Module):

  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    # Extract Feature Layers
    self.features = model.features
    # Extract Average Pooling Layer
    self.pooling = model.avgpool
    # Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
  
  def forward(self, x):
    # Take image x and return a feature vector
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    return out 

feature_extractor = FeatureExtractor(alexnet)
feature_extractor = feature_extractor.to(device)

In [10]:
print(summary(feature_extractor, input_size=(3, 224, 224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [11]:
import cv2
from torchvision import transforms
from torch import no_grad

## Extract Features for Caltech 101 Dataset

In [12]:
!pip install imutils
import tarfile
from imutils import paths
import os
import cv2
import numpy as np
from torchvision import transforms

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25861 sha256=418717f9c68fde3ad5944062d2f24a96e3472663ef03028bdeea329871e640fc
  Stored in directory: /root/.cache/pip/wheels/35/e4/69/cb99d996d14a2971b79b990d68b05a17d58ce530ff96090dfc
Successfully built imutils


In [13]:
image_paths = list(paths.list_images('/kaggle/input/vr3bdataset/101_ObjectCategories'))

images = []
labels = []

for img_path in image_paths:

    label = img_path.split(os.path.sep)[-2]
    if label == "BACKGROUND_Google":
        continue
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    images.append(img)
    labels.append(label)
    
images = np.array(images, dtype=object)
labels = np.array(labels)

In [14]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [15]:
features_array = []

for i in range(len(images)):
    img = images[i]
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with no_grad():
        img_features = feature_extractor(img)
    img_features = img_features.cpu().detach().numpy().reshape(-1)
    features_array.append(img_features)

In [16]:
np.array(features_array).shape

(8677, 9216)

## Numpy

In [17]:
import numpy as np

In [18]:
X = np.array(features_array)
y = np.array(labels)

## Pandas

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame(X)
df['label'] = y

In [21]:
df

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,scorpion
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,scorpion
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,scorpion
3,0.497246,3.570379,3.570379,0.931328,3.660758,3.660758,1.366830,3.570379,3.570379,0.639022,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,scorpion
4,1.611503,2.885494,2.342189,3.464070,6.232759,6.232759,3.825613,6.601429,6.601429,1.158655,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,scorpion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672,0.081423,0.427116,0.000000,0.000000,0.000000,0.000000,0.081423,0.427116,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.262583,0.075923,0.075923,0.0,0.00000,0.000000,electric_guitar
8673,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.398249,0.179201,0.000000,0.000000,0.000000,0.0,0.90765,0.179201,electric_guitar
8674,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.115345,3.344222,3.985552,1.248986,1.248986,0.000000,0.0,0.00000,3.985552,electric_guitar
8675,6.668174,6.668174,3.154979,0.773614,0.000000,0.000000,7.307738,3.667111,3.154979,0.773614,...,0.000000,0.000000,0.000000,0.200211,0.000000,0.000000,0.0,0.00000,0.000000,electric_guitar


In [22]:
X=df.drop("label",axis=1)
y=df["label"]
y

0              scorpion
1              scorpion
2              scorpion
3              scorpion
4              scorpion
             ...       
8672    electric_guitar
8673    electric_guitar
8674    electric_guitar
8675    electric_guitar
8676    electric_guitar
Name: label, Length: 8677, dtype: object

In [23]:
import pickle
# Open a file and use dump()
with open('X.pkl', 'wb') as file:
    pickle.dump(X, file)
    
with open('y.pkl', 'wb') as file:
    pickle.dump(y, file)

In [24]:
import pickle
  
# Open the file in binary mode
with open('X.pkl', 'rb') as file:
    # Call load method to deserialze
    X = pickle.load(file)
with open('y.pkl', 'rb') as file:
    # Call load method to deserialze
    y = pickle.load(file)

Doing train test split with the test_size =0.33

In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
X=pca.fit_transform(X)
X

array([[  8.605157  , -16.973955  ,  -0.6405154 , ...,   1.2428429 ,
         -1.3761779 ,  -3.6642547 ],
       [  7.17025   , -10.27675   ,   3.6053593 , ...,   3.235307  ,
          2.8440979 ,  -1.1443632 ],
       [-16.676764  , -25.323126  , -14.54627   , ...,   0.18158007,
          2.2176256 ,  -2.271478  ],
       ...,
       [-18.654108  ,  -6.7168403 ,  -9.131127  , ...,   2.2635539 ,
         -1.1930246 ,  -1.7692369 ],
       [  1.4904141 , -13.152138  ,  -3.4190593 , ...,  -0.08400166,
         -1.3578844 ,  -1.7414469 ],
       [ 12.132221  , -15.452811  ,  13.334198  , ...,  -1.8904772 ,
          4.999305  ,  -1.0212206 ]], dtype=float32)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape

(5813, 500)

### Applied Logistic Regression Classifier on these images

In [27]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.9155027932960894

In [28]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.865572625698324

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.7063547486033519

In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.5932262569832403

In [31]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.700768156424581

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.6962290502793296

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.74231843575419

In [34]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.7370810055865922

In [35]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.7538407821229051

In [36]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.7475558659217877

In [37]:
import pickle
  
# Open the file in binary mode
with open('X.pkl', 'rb') as file:
    # Call load method to deserialze
    X = pickle.load(file)
with open('y.pkl', 'rb') as file:
    # Call load method to deserialze
    y = pickle.load(file)
    
# from sklearn.decomposition import PCA
# pca = PCA(n_components=3000)
# X=pca.fit_transform(X)


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape

(5813, 9216)

In [39]:
pca.explained_variance_ratio_.sum()

0.7194028

In [40]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.9214385474860335

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8879189944134078

In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8348463687150838

In [43]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.630586592178771

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.6801675977653632

In [45]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.5108240223463687

In [46]:
import pickle
  
# Open the file in binary mode
with open('X.pkl', 'rb') as file:
    # Call load method to deserialze
    X = pickle.load(file)
with open('y.pkl', 'rb') as file:
    # Call load method to deserialze
    y = pickle.load(file)
    
from sklearn.decomposition import PCA
pca = PCA(n_components=3000)
X=pca.fit_transform(X)
pca.explained_variance_ratio_.sum()

0.96026766

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape

(5813, 3000)

In [48]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

0.9221368715083799

In [49]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.5429469273743017

In [50]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.3121508379888268

In [51]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

0.6452513966480447

In [52]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.5167597765363129

In [53]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.42248603351955305